# Model

<u>Notes:</u>
1. Please run the utility file to get the clean data frames
2. Due to time limitations I will only train a model to predict one product. I will choose the one with the most interesting seasonality to I can run a SARIMAX model.

__SARIMAX Model__

All the needed SARIMAX components exist in the given raw data.

**Therefore I will use Product E to demonstrate:**

![image](seasonal_decomposition_e.jpg)

### Dependencies

In [477]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

### Getting the data

In [478]:
sales_df = pd.read_pickle('sales.pkl')
holidays_df = pd.read_pickle('holidays.pkl')
events_df = pd.read_pickle('events.pkl')

Extract product e from the data set

In [479]:
cat_e_df = sales_df[sales_df['category'] == 'e'].copy()
cat_e_df = cat_e_df.drop('category', axis = 1)
cat_e_df['date'] = pd.to_datetime(cat_e_df['date'], format='%d-%m-%Y')

cat_e_df = cat_e_df.set_index('date').sort_values('date', ascending = True)

### Augmented Dickey–Fuller Test
The Augmented Dickey-Fuller Test is used to determine if time-series data is stationary or not. Similar to a t-test, we set a significance level before the test and make conclusions on the hypothesis based on the resulting p-value.

<u>Null Hypothesis:</u> The data is not stationary.

<u>Alternative Hypothesis:</u>  The data is stationary.

In [480]:
print('Results of Dickey Fuller Test:')
dftest = adfuller(cat_e_df['sales'], autolag='AIC')

dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
for key,value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
    
print(dfoutput)

Results of Dickey Fuller Test:
Test Statistic                 -0.964841
p-value                         0.765871
#Lags Used                      7.000000
Number of Observations Used    33.000000
Critical Value (1%)            -3.646135
Critical Value (5%)            -2.954127
Critical Value (10%)           -2.615968
dtype: float64


**Pvalue = 0.765 (grater then 0.05) there for we conclude the data is not stationary**

Create the Exogenous Variables data using holidays and events files:
- count the number of events per month
- complete missing months and fill na(0) where needed to match the sales date range

In [481]:
events_per_month = events_df.resample('M').size().reset_index(name='event_count')

events_per_month['date'] = events_per_month['date'].dt.to_period('M').dt.to_timestamp()

holidays_per_month = holidays_df.resample('M').size().reset_index(name='holiday_count')

holidays_per_month['date'] = holidays_per_month['date'].dt.to_period('M').dt.to_timestamp()

exog_set = pd.merge(events_per_month, holidays_per_month, on='date', how='outer').fillna(0).set_index('date')

In [482]:
date_range = pd.date_range(start='2020-01-01', end='2023-05-01', freq='MS')  # MS for start of month
exog_complete = pd.DataFrame(index=date_range)

exog_df = exog_set.merge(exog_complete, how='outer', left_index=True, right_index=True).fillna(0).reset_index().rename(columns = {'index' : 'date'})

In [483]:
exog_df = exog_df.set_index('date')

### Split to train + validation sets:
* Training period - 01-2020 - 02-2023
* Validation period - 03-2023 - 05-2023


In [484]:
train_start = pd.to_datetime('01-01-2020', dayfirst=True)
train_end = pd.to_datetime('28-02-2023', dayfirst=True)
val_start = pd.to_datetime('01-03-2023', dayfirst=True)
val_end = pd.to_datetime('31-05-2023', dayfirst=True)

In [485]:
cat_e_df.index = pd.to_datetime(cat_e_df.index)
exog_df.index = pd.to_datetime(exog_df.index)

train_df = cat_e_df[(cat_e_df.index >= pd.Timestamp(train_start)) & (cat_e_df.index <= pd.Timestamp(train_end))]
val_df = cat_e_df[(cat_e_df.index >= pd.Timestamp(val_start)) & (cat_e_df.index <= pd.Timestamp(val_end))]

exog_train_df = exog_df[(exog_df.index >= pd.Timestamp(train_start)) & (exog_df.index <= pd.Timestamp(train_end))]
exog_val_df = exog_df[(exog_df.index >= pd.Timestamp(val_start)) & (exog_df.index <= pd.Timestamp(val_end))]

In [486]:
exog_train_df = exog_train_df.reindex(train_df.index)

# SARIMAX

In [487]:
model = sm.tsa.SARIMAX(train_df, exog=exog_train_df, order=(1,1,1), seasonal_order=(1, 1, 1, 12), trend='t', trend_order=2)

model_fit = model.fit(disp=False)
print(model_fit.summary())

                                     SARIMAX Results                                      
Dep. Variable:                              sales   No. Observations:                   38
Model:             SARIMAX(1, 1, 1)x(1, 1, 1, 12)   Log Likelihood                -175.291
Date:                            Sat, 08 Jul 2023   AIC                            366.583
Time:                                    15:44:00   BIC                            376.334
Sample:                                01-01-2020   HQIC                           369.287
                                     - 02-01-2023                                         
Covariance Type:                              opg                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
drift            -1.2342      4.498     -0.274      0.784     -10.051       7.582
event_count      13.2920  

In [488]:
exog_val_rows = exog_val_df.values
predictions = []

for i in range(len(val_df)):
    prediction = model_fit.get_prediction(start=val_df.index[i], end=val_df.index[i], exog=exog_val_rows[:i+1], dynamic=True)
    predicted_value = prediction.predicted_mean[0]
    predictions.append(predicted_value)

train_df[val_df.index] = predictions

model_fit = model.fit(start_params=model_fit.params, disp=False)

In [489]:
val_df['predicted_value'] = predictions

mse = mean_squared_error(val_df['sales'], val_df['predicted_value'])
rmse = np.sqrt(mse)

## Here are some example for parameters tuning tested and their RMSE:

* RMSE: 503.8497035837966

order=(1, 1, 1), seasonal_order=(1, 1, 1, 12)

* RMSE: 475.2791161906294

order=(1,1,1), seasonal_order=(1, 1, 1, 12), trend='t', trend_order=2

* RMSE: 813.0748098260805

order=(2,1,1), seasonal_order=(2, 1, 1, 12), trend='t', trend_order=2

* RMSE: 1169.2753243202699

order=(1,2,1), seasonal_order=(1, 2, 1, 12), trend='t', trend_order=2

* RMSE: 529.7055698060717

order=(1,1,2), seasonal_order=(1, 1, 2, 12), trend='t', trend_order=2
* RMSE: 657.3187014265925

order=(0,1,0), seasonal_order=(0, 1, 0, 12), trend='t', trend_order=2


### Retrain for the entire period of data:

create a full range of the exog data

In [490]:
retrain_start = pd.to_datetime('01-01-2020', dayfirst=True)
retrain_end = pd.to_datetime('31-05-2023', dayfirst=True)
predict_start = pd.to_datetime('01-06-2023', dayfirst=True)
predict_end = pd.to_datetime('30-11-2023', dayfirst=True)

In [491]:
retrain_df = cat_e_df[(cat_e_df.index >= pd.Timestamp(retrain_start)) & (cat_e_df.index <= pd.Timestamp(retrain_end))]
exog_retrain = exog_df[(exog_df.index >= pd.Timestamp(retrain_start)) & (exog_df.index <= pd.Timestamp(retrain_end))]
exog_predict = exog_df[(exog_df.index >= pd.Timestamp(predict_start)) & (exog_df.index <= pd.Timestamp(predict_end))]

exog_retrain = exog_retrain.reindex(retrain_df.index)

In [492]:
model = sm.tsa.SARIMAX(retrain_df, exog=exog_retrain, order=(1,1,1), seasonal_order=(1, 1, 1, 12), trend='t', trend_order=2)

model_fit = model.fit(disp=False)
print(model_fit.summary())

                                     SARIMAX Results                                      
Dep. Variable:                              sales   No. Observations:                   41
Model:             SARIMAX(1, 1, 1)x(1, 1, 1, 12)   Log Likelihood                -199.719
Date:                            Sat, 08 Jul 2023   AIC                            415.438
Time:                                    15:44:02   BIC                            426.096
Sample:                                01-01-2020   HQIC                           418.696
                                     - 05-01-2023                                         
Covariance Type:                              opg                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
drift            -1.3424      2.712     -0.495      0.621      -6.658       3.973
event_count      16.0117  

In [493]:
exog_val_rows = exog_predict.values
predictions = []

for i in range(len(exog_predict)):
    prediction = model_fit.get_prediction(start=exog_predict.index[i], end=exog_predict.index[i], exog=exog_val_rows[:i+1], dynamic=True)
    predicted_value = prediction.predicted_mean[0]
    predictions.append(predicted_value)

retrain_df[exog_predict.index] = predictions

model_fit = model.fit(start_params=model_fit.params, disp=False)
predictions

[414.06350281920857,
 621.2410772131253,
 556.9253293063235,
 -306.3300175486068,
 443.0662304079341,
 702.1585540792671]

In [494]:
sales_predictions = pd.DataFrame()
sales_predictions['sales predictions'] = predictions
sales_predictions.set_index(exog_predict.index, inplace=True)
sales_predictions

,sales predictions
date,
2023-06-01,414.063503
2023-07-01,621.241077
2023-08-01,556.925329
2023-09-01,-306.330018
2023-10-01,443.066230
2023-11-01,702.158554
